## Creating an API on 1st Environment 
_Mariem Kachouri & Dinara Veshchezerova_ 

In [1]:
## Imports 
from keras.layers import Input, Embedding, Flatten, Dot
from keras.models import Model
import numpy as np 
import pickle

Using TensorFlow backend.


In [2]:
def build_model(nb_users, nb_items, embedding_size=30):
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')

    user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                               input_length=1, name='user_embedding')(user_id_input)

    item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                               input_length=1, name='item_embedding')(item_id_input)

  
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

    y = Dot(axes=1)([user_vecs, item_vecs])
    
    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer='adam', loss='MAE')

    return(model)

In [2]:
import tensorflow as tf

In [4]:
import numpy as np

In [5]:
print(np.__version__)

1.15.2


In [3]:
from flask import Flask, redirect, url_for, request, Response, jsonify, json
import requests 
import tensorflow as tf

app = Flask(__name__)
app.url_map.strict_slashes = False

In [ ]:
@app.route("/train", methods=['GET','POST'])
def train():
    
    user_history = [int(user) for user  in request.args.getlist('user_history')]
    item_history = [int(item) for item in request.args.getlist('item_history')]
    rating_history = [int(rating) for rating in request.args.getlist('rating_history')]
    nb_users = int(request.args.get('nb_users'))
    nb_items = int(request.args.get('nb_items'))
    

    global model,graph
    model = build_model(nb_users,nb_items,30)
    model.fit([user_history, item_history], rating_history,
                batch_size=64, 
                epochs=20, 
                validation_split=0.1,
                shuffle=True)
     
    graph = tf.get_default_graph()
    #pickle.dump(model, open('model.pkl', 'wb'))
 
    return 'Model trained.'

@app.route("/predict", methods=['GET','POST'])
def predict():
    with graph.as_default():
        
        next_user = int(request.args.get('next_user'))
        next_item = int(request.args.get('next_item'))

        rating_predicted = model.predict([np.array(next_user).reshape(1,1), np.array(next_item).reshape(1,1)])[0][0]

        return jsonify({'rating':str(round(rating_predicted))})

app.run(host='127.0.0.1', port=80)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)


Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 2s 169us/step - loss: 2.8970 - val_loss: 2.7901
Epoch 2/20
9000/9000 [==============================] - 0s 21us/step - loss: 2.8230 - val_loss: 2.5717
Epoch 3/20
9000/9000 [==============================] - 0s 20us/step - loss: 2.2340 - val_loss: 1.6384
Epoch 4/20
9000/9000 [==============================] - 0s 20us/step - loss: 1.3035 - val_loss: 0.9067
Epoch 5/20
9000/9000 [==============================] - 0s 22us/step - loss: 0.7559 - val_loss: 0.6169
Epoch 6/20
9000/9000 [==============================] - 0s 21us/step - loss: 0.5967 - val_loss: 0.5803
Epoch 7/20
9000/9000 [==============================] - 0s 21us/step - loss: 0.5751 - val_loss: 0.5737
Epoch 8/20
9000/9000 [==============================] - 0s 20us/step - loss: 0.5686 - val_loss: 0.5709
Epoch 9/20
9000/9000 [==============================] - 0s 20us/step - loss: 0.5618 - val_loss: 0.5669
Epoch 10/20
9000/9000 [=

127.0.0.1 - - [23/Feb/2019 17:13:20] "POST /train?user_history=27&user_history=10&user_history=62&user_history=1&user_history=47&user_history=30&user_history=71&user_history=77&user_history=76&user_history=34&user_history=78&user_history=7&user_history=88&user_history=68&user_history=26&user_history=75&user_history=52&user_history=67&user_history=51&user_history=50&user_history=68&user_history=93&user_history=17&user_history=30&user_history=45&user_history=0&user_history=53&user_history=69&user_history=35&user_history=14&user_history=73&user_history=26&user_history=1&user_history=33&user_history=32&user_history=50&user_history=80&user_history=52&user_history=16&user_history=3&user_history=76&user_history=33&user_history=52&user_history=61&user_history=0&user_history=8&user_history=96&user_history=23&user_history=14&user_history=22&user_history=26&user_history=98&user_history=70&user_history=20&user_history=79&user_history=24&user_history=86&user_history=17&user_history=89&user_history=

127.0.0.1 - - [23/Feb/2019 17:13:25] "POST /predict?next_item=10&next_user=5 HTTP/1.1" 200 -


In [ ]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=80)